In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome() # Launch Chrome browser

driver.get("https://hoopshype.com/salaries/players") # Navigate to our main page
html = driver.page_source # Get the source code
soup = BeautifulSoup(html, 'html.parser') # Represent the source code & Allow us to do searching

# Locate the salary table & extract all links inside it
salary_table = soup.find("table", {"class": ("hh-salaries-ranking-table", 
                                             "hh-salaries-table-sortable",
                                             "responsive")}
                        )
# players_basic_information_links store all links
players_basic_information_links  = [
    a_tag_element.get("href") 
    for a_tag_element in salary_table.findChildren("a" , recursive=True)
]

In [3]:
# Initiate arrays to store information
all_players_info = []

In [4]:
for basic_information_page_link in players_basic_information_links[:2]:
    driver.get(basic_information_page_link ) # Navigate to basic information page
    basic_information_html = driver.page_source # Get the source code
    soup = BeautifulSoup(basic_information_html, 'html.parser') # Represent the source code & Allow us to do searching

    # Extract Information
    player_name = soup.find("div", {"class": "player-fullname"}).text.strip() # Get Name
    player_jersey = soup.find("div", {"class": "player-jersey"}).text.strip() # Get Jersey
    player_team = soup.find("div", {"class": "player-team"}).text.strip() # Get Team

    # Get all elements with player-bio-text-line-value
    # Match each element to corresponding biography information
    bio_text_values = soup.find_all("span", {"class": "player-bio-text-line-value"})
    player_position = bio_text_values[0].text.strip()
    player_birthdate = bio_text_values[1].text.strip()
    player_height = bio_text_values[2].text.strip()
    player_weight = bio_text_values[3].text.strip()
    player_salary = bio_text_values[4].text.strip()
    
    
    # Store information
    all_players_info.append([
        player_name, 
        player_jersey, 
        player_team, 
        player_position, 
        player_birthdate, 
        player_height, 
        player_weight, 
        player_salary
    ])
    # Pause 5 seconds before we get information for another player
    # Otherwise, we increase the load for their server significantly
    time.sleep(5)    

In [5]:
# Create a dataframe, you may treat it as a table
df = pd.DataFrame(all_players_info, 
                  columns =[
                      "Name", 
                      "Jersey", 
                      "Team", 
                      "Position", 
                      "Birthdate", 
                      "Height", 
                      "Weight", 
                      "Salary"
                  ]
                 )


In [6]:
df.to_csv(".\players.csv",index=False)